<a href="https://colab.research.google.com/github/sam505/Machine_Learning/blob/master/PYTORCH_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import torchvision
import torch
import torch.nn as nn
from torch import optim
import torch.utils.data
import torch.nn.functional as F
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

In [2]:
transform = transforms.Compose([
                 transforms.ToTensor (), # Convert to Tensor
                 transforms.Normalize ((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), # normalize
    ])


train_batch_size = 64
test_batch_size = 128

cifar_train = torchvision.datasets.CIFAR10(root="~/", 
                                         download=True, 
                                         transform=transform, 
                                         train=True)

cifar_test = torchvision.datasets.CIFAR10(root="~/", 
                                        download=True, 
                                        transform=transform, 
                                        train=False)

train_loader = torch.utils.data.DataLoader(cifar_train,
                                          shuffle=True,
                                          batch_size=train_batch_size,
                                          num_workers=2)
test_loader = torch.utils.data.DataLoader(cifar_test,
                                          shuffle=False,
                                          batch_size=test_batch_size,
                                          num_workers=2)


Extracting /root/cifar-10-python.tar.gz to /root/
Files already downloaded and verified


In [3]:
len(cifar_train)

50000

In [4]:
len(cifar_test)

10000

In [5]:
learning_rates = [0.1, 0.01, 0.001]

In [6]:
class LeNet (nn.Module): # network structure
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
 
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [7]:
def train(model, optimizer, epoch, train_loader, log_interval):
    # State that you are training the model
    model.train()
    correct = 0
    count = 1
    # define loss function
    loss_fn = nn.CrossEntropyLoss()

    # Iterate over batches of data
    for batch_idx, (data, target) in enumerate(train_loader):
        # Wrap the input and target output in the `Variable` wrapper
        data, target = Variable(data), Variable(target)

        check_gpu = torch.cuda.is_available()
        if check_gpu:
            data = data.to("cuda")
            target = target.to("cuda")

        # Clear the gradients, since PyTorch accumulates them
        optimizer.zero_grad()

        # Forward propagation
        output = model(data)

        loss = loss_fn(output, target)

        # Backward propagation
        loss.backward()

        # Update the parameters(weight,bias)
        optimizer.step()
        
        predicted = torch.max(output.data, 1)[1] 
        correct += (predicted == target).sum()

        # print log
        if batch_idx % log_interval == 0:
            train_accuracy = correct / ((batch_idx+1)*len(data))
            train_loss = loss.data.item()
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                train_loss, train_accuracy))
        
    return loss.data.item(), train_accuracy

In [8]:
def test(model, epoch, test_loader):
    # State that you are testing the model; this prevents layers e.g. Dropout to take effect
    model.eval()

    # Init loss & correct prediction accumulators
    test_loss = 0
    correct = 0

    # define loss function
    loss_fn = nn.CrossEntropyLoss(size_average=False)

    # Iterate over data
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)

        check_gpu = torch.cuda.is_available()
        if check_gpu:
            data = data.to("cuda")
            target = target.to("cuda")

        # Forward propagation
        output = model(data)

        # Calculate & accumulate loss
        test_loss += loss_fn(output, target).data.item()

        # Get the index of the max log-probability (the predicted output label)
        pred = np.argmax(output.cpu().data, axis=1)

        # If correct, increment correct prediction accumulator
        correct = correct + np.equal(pred, target.cpu().data).sum()

    # Print log
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    return test_loss, correct / len(test_loader.dataset)

In [9]:
# Provide seed for the pseudorandom number generator s.t. the same results can be reproduced
for learning_date in learning_rates:
    torch.manual_seed(123)

    model = LeNet()

    check_gpu = torch.cuda.is_available()
    if check_gpu:
        model = model.cuda()
        print ('USING GPU')
    else:
        print ('USING CPU')
    epochs = 10
    log_interval = 100
    writer = SummaryWriter("runs/CIFAR/Adam_{}".format(learning_date))
    optimizer = optim.Adam(model.parameters(), lr=learning_date)
    for epoch in range(1, epochs + 1):
        train_loss, train_acc = train(model, optimizer, epoch, train_loader, log_interval=log_interval)
        test_loss, test_acc = test(model, epoch, test_loader) 
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/test', test_loss, epoch)
        writer.add_scalar('Accuracy/train', train_acc, epoch)
        writer.add_scalar('Accuracy/test', test_acc, epoch)

USING GPU
Train set, Epoch 1 [0/50000 (0%)]	Loss: 2.296416	Accuracy: 0.12
Train set, Epoch 1 [6400/50000 (13%)]	Loss: 2.297132	Accuracy: 0.10
Train set, Epoch 1 [12800/50000 (26%)]	Loss: 2.295130	Accuracy: 0.10
Train set, Epoch 1 [19200/50000 (38%)]	Loss: 2.310893	Accuracy: 0.10
Train set, Epoch 1 [25600/50000 (51%)]	Loss: 2.337512	Accuracy: 0.10
Train set, Epoch 1 [32000/50000 (64%)]	Loss: 2.335846	Accuracy: 0.10
Train set, Epoch 1 [38400/50000 (77%)]	Loss: 2.293926	Accuracy: 0.10
Train set, Epoch 1 [44800/50000 (90%)]	Loss: 2.305306	Accuracy: 0.10


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 2.3221, Accuracy: 1000/10000 (10.00%)

Train set, Epoch 2 [0/50000 (0%)]	Loss: 2.295189	Accuracy: 0.17
Train set, Epoch 2 [6400/50000 (13%)]	Loss: 2.307580	Accuracy: 0.10
Train set, Epoch 2 [12800/50000 (26%)]	Loss: 2.314214	Accuracy: 0.10
Train set, Epoch 2 [19200/50000 (38%)]	Loss: 2.299665	Accuracy: 0.10
Train set, Epoch 2 [25600/50000 (51%)]	Loss: 2.314833	Accuracy: 0.10
Train set, Epoch 2 [32000/50000 (64%)]	Loss: 2.322727	Accuracy: 0.10
Train set, Epoch 2 [38400/50000 (77%)]	Loss: 2.294224	Accuracy: 0.10
Train set, Epoch 2 [44800/50000 (90%)]	Loss: 2.318937	Accuracy: 0.10

Test set, Epoch 2 , Average loss: 2.3068, Accuracy: 1000/10000 (10.00%)

Train set, Epoch 3 [0/50000 (0%)]	Loss: 2.324815	Accuracy: 0.05
Train set, Epoch 3 [6400/50000 (13%)]	Loss: 2.302771	Accuracy: 0.10
Train set, Epoch 3 [12800/50000 (26%)]	Loss: 2.306137	Accuracy: 0.10
Train set, Epoch 3 [19200/50000 (38%)]	Loss: 2.302993	Accuracy: 0.10
Train set, Epoch 3 [25600/50000 (51%